# Análise de Receitas x Despesas Candidatos do Paraná nas Eleições 2022

O objetivo do presente estudo é obter insights relevantes após analisar os dados públicos de prestação de contas dos partidos políticos
durante as eleições de 2022 no estado do Paraná.

**Fonte dos Dados: ** https://dados.gov.br/dados/conjuntos-dados/dadosabertos-tse-jus-br-dataset-prestacao-de-contas-eleitorais-2022

A semântica e clareza dos insights vai depender da qualidade e veracidade da fonte de dados.

## Lendo dados de Receitas e Despesas (DataSets)

In [1]:
import pandas as pd

receitas = pd.read_csv('receitas_candidatos_2022_PR.csv', encoding='latin1', sep=';', parse_dates=['DT_GERACAO','HH_GERACAO','DT_PRESTACAO_CONTAS','DT_RECEITA'], dayfirst=True, decimal=',', thousands='.')
despesas = pd.read_csv('despesas_contratadas_candidatos_2022_PR.csv', encoding='latin1', sep=';', parse_dates=['DT_GERACAO','HH_GERACAO','DT_PRESTACAO_CONTAS','DT_DESPESA'], dayfirst=True, decimal=',', thousands='.')

## Receitas x Despesas por Partido

In [2]:

receitas_partido = receitas[['NM_PARTIDO','VR_RECEITA']].groupby('NM_PARTIDO').sum().sort_values(by='VR_RECEITA', ascending=False)
despesas_partido = despesas[['NM_PARTIDO','VR_DESPESA_CONTRATADA']].groupby('NM_PARTIDO').sum().sort_values(by='VR_DESPESA_CONTRATADA', ascending=False)

fluxos_partidos = pd.merge(receitas_partido, despesas_partido, on='NM_PARTIDO')
fluxos_partidos['SALDO'] = fluxos_partidos['VR_RECEITA'] - fluxos_partidos['VR_DESPESA_CONTRATADA']
fluxos_partidos['PERCENTUAL_SALDO'] = round(fluxos_partidos['SALDO'] / fluxos_partidos['VR_RECEITA'] * 100, 2)
fluxos_partidos = fluxos_partidos.sort_values(by='PERCENTUAL_SALDO', ascending=False)

display(fluxos_partidos)

,VR_RECEITA,VR_DESPESA_CONTRATADA,SALDO,PERCENTUAL_SALDO
NM_PARTIDO,,,,
Partido da Causa Operária,48363.64,0.00,48363.64,100.00
Rede Sustentabilidade,2788285.92,1544448.57,1243837.35,44.61
Solidariedade,8367722.57,6222528.30,2145194.27,25.64
Partido da Mulher Brasileira,352594.36,274022.51,78571.85,22.28
AGIR,1273068.04,1109934.69,163133.35,12.81
Partido Trabalhista Brasileiro,3738782.75,3265624.17,473158.58,12.66
Partido Socialista dos Trabalhadores Unificado,4042.00,3541.00,501.00,12.39
Podemos,15673984.95,14064294.13,1609690.82,10.27
Partido Comunista do Brasil,330292.10,297745.91,32546.19,9.85


**Na tabela acima alguns pontos chamam atenção:**

- Pelo menos um partido não prestou contas de como gastou as receitas arrecadas;
- Pelo menos um partido prestou contas de 100% de como gastou as receitas arrecadadas;
- A maioria dos partidos ficou com saldo positivo quando se diminui o valor das receitas pelos valores de despesas na prestação de contas;
- Por qual razão a maioria dos partidos ficaram com saldo positivo durante o período eleitoral?
      (Pergunta não pode ser respondida apenas com os dados, é uma pergunta para reflexão, precisamos buscar entender negocialmente porque a maioria
      dos partidos deixou de prestar contas de alguma porcentagem de toda receita arrecadada no período eleitoral)

Vamos continuar examinando os dados e aos poucos consildar o presente estudo.

## Detalhando Fontes de Receitas dos Partidos

In [3]:

fontes_receitas = receitas[['DS_FONTE_RECEITA','VR_RECEITA']].groupby('DS_FONTE_RECEITA').sum()
display(fontes_receitas) 

,VR_RECEITA
DS_FONTE_RECEITA,
FUNDO ESPECIAL,1.974295e+08
FUNDO PARTIDARIO,2.610100e+07
OUTROS RECURSOS,7.559484e+07
